In [2]:
!pip install pycaret-ts-alpha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.1/468.1 kB 626.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.6 MB/s eta 0:00:0000:0100:01
  Created wheel for pyod: filename=pyod-1.0.4-py3-none-any.whl size=165073 sha256=da751320b2bb912670d7f903cf2b207ac4f5bbc65c1a8288078f8363b1ce77a2
  Stored in directory: /root/.cache/pip/wheels/3d/48/a8/87d61660791c7e6b0f5c3068da7fb17ade7fdc041e864fe053
Successfully built pyod


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
from random import sample
import plotly.express as px
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import probplot, boxcox
from scipy.special import inv_boxcox
from scipy import stats
from pycaret.time_series import *

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)

In [5]:
train = pd.read_csv("../input/tabular-playground-series-sep-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-sep-2022/test.csv")
sub = pd.read_csv("../input/tabular-playground-series-sep-2022/sample_submission.csv")

# Creating the concatenated group to forecast
train['ID'] = train['store'] + '-' + train['product'] + '-' + train['country']
test['ID'] = test['store'] + '-' + test['product'] + '-' + test['country']
train.drop(columns=["country","store","product"],inplace=True)
test.drop(columns=["country","store","product"],inplace=True)
# Split data
training_set = train[train['date'] < '2020-01-01']
validation_set = train[train['date'] >= '2020-01-01']

In [7]:
result_df = pd.DataFrame()
object_list = list(train['ID'].unique())
for i in tqdm(range(48)):
    #Status
    status = 100*(i/len(object_list))
    print(f"Progressing status: ",\
          (int(status/10)*"="),\
          f"{status} %")
    # Create unique dataset
    unique_df = training_set[training_set.ID == training_set['ID'].unique()[i]]
    
    # Save index and unique group name
    row_line = unique_df.row_id
    row_ID = training_set['ID'].unique()[i]
    # We dont need anymore drop them all
    unique_df = unique_df.drop(columns=["row_id","ID"])
    
    # for validation dataset
    
    val_df_unique1 = unique_df[unique_df["date"].str.contains("2019")].reset_index(drop=True)
    train_val_df_unique = unique_df[~unique_df["date"].str.contains("2019")]
    
    # transforming for pycaret 
    train_val_df_unique['date'] = pd.DatetimeIndex(train_val_df_unique['date'])
    train_val_df_unique = train_val_df_unique.set_index(train_val_df_unique["date"]).drop(columns="date")
        
    # Setup for pycaret
    setup(data = train_val_df_unique,
          target = "num_sold",
          verbose=False,
          index=None,
          ignore_features=None,
          #transform_target="box-cox", # “box-cox”, “log”, “sqrt”, “exp”, “cos”,
          #scale_target = "robust", # “zscore”, “minmax”, “maxabs”, “robust”
          fold_strategy = "expanding", # "rolling","sliding"
          fold = 3, # Default
          fh = 12,# Default: 1
         )
    # Tuning of Hyperparamters
    tuned_prophet = tune_model(create_model('prophet'),verbose=False)
    result = predict_model(tuned_prophet, fh = 365) 
    
    result["index"] = row_line[-365:].values
    result.sort_values(by="index",inplace=True)
    result_df = pd.concat([result_df,result])


train_values = train[train.row_id.isin(result_df["index"].values)].num_sold.values
result_df = result_df.sort_values(by="index") 
pred = result_df.y_pred.values


,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2018-11-13,7.5307,8.7739,0.0752,0.0719,1.0404,0.8225,-0.6919
1,2018-11-25,3.9320,4.6868,0.0398,0.0390,0.5423,0.4400,0.4852
2,2018-12-07,10.9647,13.6110,0.0974,0.1024,1.5165,1.2841,-0.1758
Mean,NaT,7.4758,9.0239,0.0708,0.0711,1.0331,0.8488,-0.1275
SD,NaT,2.8714,3.6476,0.0237,0.0259,0.3977,0.3451,0.4818


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


100%|██████████| 48/48 [1:05:35<00:00, 81.98s/it]

RMSE METRIC FOR 2020: 3481.62
MAPE METRIC FOR 2020: 21.18 %
SMAPE METRIC FOR 2020: 22.47%


In [19]:
models()

,Name,Reference,Turbo
ID,,,
naive,Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
grand_means,Grand Means Forecaster,sktime.forecasting.naive.NaiveForecaster,True
snaive,Seasonal Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
polytrend,Polynomial Trend Forecaster,sktime.forecasting.trend.PolynomialTrendForeca...,True
arima,ARIMA,sktime.forecasting.arima.ARIMA,True
auto_arima,Auto ARIMA,sktime.forecasting.arima.AutoARIMA,True
exp_smooth,Exponential Smoothing,sktime.forecasting.exp_smoothing.ExponentialSm...,True
croston,Croston,sktime.forecasting.croston.Croston,True
ets,ETS,sktime.forecasting.ets.AutoETS,True


In [40]:
# Create empty DataFrame for concatenate

result_df = pd.DataFrame()

# We created unique attributes, we will iterating with this list
object_list = list(train['ID'].unique())

for i in tqdm(range(48)):
    # Create unique dataset by unique ID

    unique_df = train[train.ID == train['ID'].unique()[i]]
        
    # for validation dataset - before the submission I choose 2020 for trying this code.

    val_df = unique_df[unique_df["date"] >= '2020-01-01']#.str.contains("2020")].reset_index(drop=True)
    train_df = unique_df[unique_df["date"] < '2020-01-01']#.str.contains("2020")]
    
    # Maybe here is false but we expecting to 366 prediction values for each unique_df
    # Shortyly I assumed that for predicting the forecasting horizon for this case
    result_lenght = val_df.shape[0]
    
    # When setting 365, the setup module returns an error about size, I'm also trying 1,3,7,15,25
    setup_fh = 28

    # Setup for pycaret
    setup(data = train_df,
          target = "num_sold",
          verbose=False, #--> Another BUG but it is okay
          index="date", # Very clever think, I don't need transforming with pd.datetime, index etc.
          ignore_features= ["row_id","ID"], # Also clever
          scale_target = "robust",
          transform_target= "box-cox", # Very nice 
          fold = 1, # For the time saving for me :D
          fh = setup_fh, # Default: 1, But I want to modular function because of I will create def with this code
          session_id = 42) # mean of universe)
    # Maybe best way, compare models and choose best 3 and blend with them all. But I want to use prophet
    
    # Create model
    model = create_model('lr_cds_dt',
                         cross_validation=True,
                         fold=1,
                         verbose=False)
    # Tuning of Hyperparamters
    
    tuned_model = tune_model(estimator = model,
                             fold=1,
                             optimize="SMAPE",
                             search_algorithm="random",
                             choose_better=True,
                             verbose=False)
    
    # Finalized experiment 
    
    finalized_model = finalize_model(tuned_model)
    
    # Prediction
    
    result = predict_model(finalized_model, fh = result_lenght)
    
    #result = result[setup_fh:]
   
    # I need for indexing and verifying
    result["index"] = val_df['row_id'].values
    result["ID"] = val_df['ID'].values
    result["date"] = val_df['date'].values
    result.sort_values(by="index",inplace=True)
    
    # After concatenating with iter line
    result_df = pd.concat([result_df,result])

  0%|          | 0/48 [00:00<?, ?it/s]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


  2%|▏         | 1/48 [00:13<10:57, 13.98s/it]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


  4%|▍         | 2/48 [00:21<07:52, 10.27s/it]

...

 96%|█████████▌| 46/48 [06:05<00:15,  7.79s/it]/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any no

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


 98%|█████████▊| 47/48 [06:13<00:07,  7.85s/it]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight 

In [41]:
result.isna().sum()

y_pred    233
index       0
ID          0
date        0
dtype: int64

In [42]:
# Create empty DataFrame for concatenate

result_df = pd.DataFrame()

# We created unique attributes, we will iterating with this list
object_list = list(train['ID'].unique())

for i in tqdm(range(48)):
    # Create unique dataset by unique ID

    unique_df = train[train.ID == train['ID'].unique()[i]]
        
    # for validation dataset - before the submission I choose 2020 for trying this code.

    val_df = unique_df[unique_df["date"] >= '2020-01-01']#.str.contains("2020")].reset_index(drop=True)
    train_df = unique_df[unique_df["date"] < '2020-01-01']#.str.contains("2020")]
    
    # Maybe here is false but we expecting to 366 prediction values for each unique_df
    # Shortyly I assumed that for predicting the forecasting horizon for this case
    result_lenght = val_df.shape[0]
    
    # When setting 365, the setup module returns an error about size, I'm also trying 1,3,7,15,25
    setup_fh = 28

    # Setup for pycaret
    setup(data = train_df,
          target = "num_sold",
          verbose=False, #--> Another BUG but it is okay
          index="date", # Very clever think, I don't need transforming with pd.datetime, index etc.
          ignore_features= ["row_id","ID"], # Also clever
          ###########################################################################
          ###########################################################################
          ###########################################################################
          #scale_target = "robust",#### I deleted on here I THINK PROBLEM IN HERE!!!!
          ###########################################################################
          ###########################################################################
          ###########################################################################
          transform_target= "box-cox", # Very nice 
          fold = 1, # For the time saving for me :D
          fh = setup_fh, # Default: 1, But I want to modular function because of I will create def with this code
          session_id = 42) # mean of universe)
    # Maybe best way, compare models and choose best 3 and blend with them all. But I want to use prophet
    
    # Create model
    model = create_model('lr_cds_dt',
                         cross_validation=True,
                         fold=1,
                         verbose=False)
    # Tuning of Hyperparamters
    
    tuned_model = tune_model(estimator = model,
                             fold=1,
                             optimize="SMAPE",
                             search_algorithm="random",
                             choose_better=True,
                             verbose=False)
    
    # Finalized experiment 
    
    finalized_model = finalize_model(tuned_model)
    
    # Prediction
    
    result = predict_model(finalized_model, fh = result_lenght)
    
    #result = result[setup_fh:]
   
    # I need for indexing and verifying
    result["index"] = val_df['row_id'].values
    result["ID"] = val_df['ID'].values
    result["date"] = val_df['date'].values
    result.sort_values(by="index",inplace=True)
    
    # After concatenating with iter line
    result_df = pd.concat([result_df,result])

  0%|          | 0/48 [00:00<?, ?it/s]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


  2%|▏         | 1/48 [00:07<05:38,  7.20s/it]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


  4%|▍         | 2/48 [00:14<05:24,  7.05s/it]/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwar

...

 96%|█████████▌| 46/48 [05:18<00:13,  6.99s/it]

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


 98%|█████████▊| 47/48 [05:25<00:06,  6.97s/it]/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in versio

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


100%|██████████| 48/48 [05:32<00:00,  6.93s/it]


In [43]:
result.isna().sum()

y_pred    0
index     0
ID        0
date      0
dtype: int64

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave 

# I was thinking the other day that the problem might be related to transform_target. Today I saw that the problem is actually related to scale_target. Scaling Target may not actually be appropriate. I couldn't understand why I was using this. Most of all, thank you very much for developing pycaret.